In [10]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging
import time

import ccxt

import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hio as hio
import helpers.hprint as hprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

>>ENV<<: is_inside_container=True: code_version=None, container_version=amp-1.1.1, is_inside_docker=True, is_inside_ci=False, CI_defined=True, CI=''
>>ENV<<: AM_AWS_PROFILE=True AM_ECR_BASE_PATH=True AM_S3_BUCKET=True AM_TELEGRAM_TOKEN=True AWS_ACCESS_KEY_ID=True AWS_DEFAULT_REGION=True AWS_SECRET_ACCESS_KEY=True GH_ACTION_ACCESS_TOKEN=True
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='None' != container_version='amp-1.1.1'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
# Git
    branch_name='CMTask13_ccxt_historical_data'
    hash='c8142941'
    # Last commits:
      * c8142941 danya    CMTask13: remove sandbox notebook                                 (  18 hours ago) Thu Sep 2 18:29:12 2021  (HEAD -> CMTask13_ccxt_historical_data

In [12]:
ALL_EXCHANGES = [
    "binance",
    "coinbase",
    "kraken",
    "huobi",
    "ftx",
    "kucoin",
    "bitfinex",
    "gateio",
    # "binanceus" # no API access for these three exchanges.
    # "bithumb"
    # "bitstamp"
]

In [14]:
credentials = hio.from_json("API_keys.json")

## Functions

These are the functions introduced in `CMTask12_CCXT_historical_data_access.ipynb`

In [128]:
def log_into_exchange(exchange_id: str):
    """
    Log into exchange via ccxt.
    """
    credentials = hio.from_json("API_keys.json")
    hdbg.dassert_in(exchange_id, credentials, msg="%s exchange ID not correct.")
    credentials = credentials[exchange_id]
    credentials["rateLimit"] = True
    exchange_class = getattr(ccxt, exchange_id)
    exchange = exchange_class(credentials)
    hdbg.dassert(
        exchange.checkRequiredCredentials(),
        msg="Required credentials not passed.",
    )
    return exchange


def describe_exchange_data(exchange_id: str):
    """ """
    exchange = log_into_exchange(exchange_id)
    print("%s:" % exchange_id)
    print("Has fetchOHLCV: %s" % exchange.has["fetchOHLCV"])
    print("Has fetchTrades: %s" % exchange.has["fetchTrades"])
    print("Available timeframes:")
    print(exchange.timeframes)
    print("Available currency pairs:")
    print(exchange.load_markets().keys())
    print("=" * 50)
    return None


def download_ohlcv_data(
    exchange_id,
    start_date,
    end_date,
    curr_symbol,
    timeframe="1m",
    step=500,
    sleep_time=3,
):
    """
    Download historical OHLCV data for given time period and currency.
    """
    exchange = log_into_exchange(exchange_id)
    hdbg.dassert_in(timeframe, exchange.timeframes)
    hdbg.dassert(exchange.has["fetchOHLCV"])
    hdbg.dassert_in(curr_symbol, exchange.load_markets().keys())
    start_date = exchange.parse8601(start_date)
    end_date = exchange.parse8601(end_date)
    # Convert to ms.
    duration = exchange.parse_timeframe(timeframe) * 1000
    all_candles = []
    for t in range(start_date, end_date + duration, duration * step):
        candles = exchange.fetch_ohlcv(curr_symbol, timeframe, t, step)
        print("Fetched", len(candles), "candles")
        if candles:
            print(
                "From",
                exchange.iso8601(candles[0][0]),
                "to",
                exchange.iso8601(candles[-1][0]),
            )
        all_candles += candles
        total_length = len(all_candles)
        print("Fetched", total_length, "candles in total")
        time.sleep(sleep_time)
    return all_candles


def download_trade_data(
    exchange_id,
    start_date,
    end_date,
    curr_symbol,
    timeframe="1m",
    step=500,
    sleep_time=3,
):
    """
    Download historical data for given time period and currency.
    """
    exchange = log_into_exchange(exchange_id)
    hdbg.dassert_in(timeframe, exchange.timeframes)
    hdbg.dassert(exchange.has["fetchTrades"])
    hdbg.dassert_in(curr_symbol, exchange.load_markets().keys())
    start_date = exchange.parse8601(start_date)
    end_date = exchange.parse8601(end_date)
    latest_trade = start_date
    all_trades = []
    while latest_trade <= end_date:
        trades = exchange.fetch_trades(
            curr_symbol,
            since=latest_trade,
            limit=step,
            params={"endTime": latest_trade + 36000},
        )
        print("Fetched", len(trades), "trades")
        if trades:
            print(
                "From",
                exchange.iso8601(trades[0]["timestamp"]),
                "to",
                exchange.iso8601(trades[-1]["timestamp"]),
            )
            latest_trade = trades[-1]["timestamp"]
        all_trades += trades
        total_length = len(all_trades)
        print("Fetched", total_length, "trades in total")
        time.sleep(sleep_time)
    return all_trades

## Check availability of historical data for exchanges

In [127]:
credentials = hio.from_json("API_keys.json")

In [129]:
for e in ALL_EXCHANGES:
    describe_exchange_data(e)

binance:
Has fetchOHLCV: True
Has fetchTrades: True
Available timeframes:
{'1m': '1m', '3m': '3m', '5m': '5m', '15m': '15m', '30m': '30m', '1h': '1h', '2h': '2h', '4h': '4h', '6h': '6h', '8h': '8h', '12h': '12h', '1d': '1d', '3d': '3d', '1w': '1w', '1M': '1M'}
Available currency pairs:
dict_keys(['ETH/BTC', 'LTC/BTC', 'BNB/BTC', 'NEO/BTC', 'QTUM/ETH', 'EOS/ETH', 'SNT/ETH', 'BNT/ETH', 'BCC/BTC', 'GAS/BTC', 'BNB/ETH', 'BTC/USDT', 'ETH/USDT', 'HSR/BTC', 'OAX/ETH', 'DNT/ETH', 'MCO/ETH', 'ICN/ETH', 'MCO/BTC', 'WTC/BTC', 'WTC/ETH', 'LRC/BTC', 'LRC/ETH', 'QTUM/BTC', 'YOYOW/BTC', 'OMG/BTC', 'OMG/ETH', 'ZRX/BTC', 'ZRX/ETH', 'STRAT/BTC', 'STRAT/ETH', 'SNGLS/BTC', 'SNGLS/ETH', 'BQX/BTC', 'BQX/ETH', 'KNC/BTC', 'KNC/ETH', 'FUN/BTC', 'FUN/ETH', 'SNM/BTC', 'SNM/ETH', 'NEO/ETH', 'IOTA/BTC', 'IOTA/ETH', 'LINK/BTC', 'LINK/ETH', 'XVG/BTC', 'XVG/ETH', 'SALT/BTC', 'SALT/ETH', 'MDA/BTC', 'MDA/ETH', 'MTL/BTC', 'MTL/ETH', 'SUB/BTC', 'SUB/ETH', 'EOS/BTC', 'SNT/BTC', 'ETC/ETH', 'ETC/BTC', 'MTH/BTC', 'MTH/ETH', 

dict_keys(['BTC/AED', 'BTC/AFN', 'BTC/ALL', 'BTC/AMD', 'BTC/ANG', 'BTC/AOA', 'BTC/ARS', 'BTC/AUD', 'BTC/AWG', 'BTC/AZN', 'BTC/BAM', 'BTC/BBD', 'BTC/BDT', 'BTC/BGN', 'BTC/BHD', 'BTC/BIF', 'BTC/BMD', 'BTC/BND', 'BTC/BOB', 'BTC/BRL', 'BTC/BSD', 'BTC/BTN', 'BTC/BWP', 'BTC/BYN', 'BTC/BYR', 'BTC/BZD', 'BTC/CAD', 'BTC/CDF', 'BTC/CHF', 'BTC/CLF', 'BTC/CLP', 'BTC/CNH', 'BTC/CNY', 'BTC/COP', 'BTC/CRC', 'BTC/CUC', 'BTC/CVE', 'BTC/CZK', 'BTC/DJF', 'BTC/DKK', 'BTC/DOP', 'BTC/DZD', 'BTC/EGP', 'BTC/ERN', 'BTC/ETB', 'BTC/EUR', 'BTC/FJD', 'BTC/FKP', 'BTC/GBP', 'BTC/GBX', 'BTC/GEL', 'BTC/GGP', 'BTC/GHS', 'BTC/GIP', 'BTC/GMD', 'BTC/GNF', 'BTC/GTQ', 'BTC/GYD', 'BTC/HKD', 'BTC/HNL', 'BTC/HRK', 'BTC/HTG', 'BTC/HUF', 'BTC/IDR', 'BTC/ILS', 'BTC/IMP', 'BTC/INR', 'BTC/IQD', 'BTC/ISK', 'BTC/JEP', 'BTC/JMD', 'BTC/JOD', 'BTC/JPY', 'BTC/KES', 'BTC/KGS', 'BTC/KHR', 'BTC/KMF', 'BTC/KRW', 'BTC/KWD', 'BTC/KYD', 'BTC/KZT', 'BTC/LAK', 'BTC/LBP', 'BTC/LKR', 'BTC/LRD', 'BTC/LSL', 'BTC/LYD', 'BTC/MAD', 'BTC/MDL', 'BTC/MGA',

dict_keys(['BCH/HUSD', 'UIP/USDT', 'STN/ETH', 'HBAR/USDT', 'QASH/ETH', 'RAI/USDT', 'MX/HT', 'WNXM/BTC', 'TOS/BTC', 'BCH3L/USDT', 'YFII/BTC', 'EOS/ETH', 'APN/BTC', 'SWRV/BTC', 'DHT/ETH', 'HBC/BTC', 'BTS/BTC', 'POWR/ETH', 'TNB/USDT', 'NEXO/USDT', 'ATP/BTC', 'DTA/BTC', 'CHR/USDT', 'LRC/BTC', 'APPC/ETH', 'PORTAL/BTC', 'XTZ/HUSD', 'FORTH/ETH', 'EVX/ETH', 'IDT/BTC', 'AAC/USDT', 'MASS/USDT', 'GSC/ETH', 'AE/ETH', 'RVN/HT', 'STPT/USDT', 'EDU/BTC', 'XEM/USDT', 'AR/USDT', 'DCR/ETH', 'YCC/BTC', 'DASH/HT', 'NULS/USDT', 'AST/ETH', 'VET/HUSD', 'ZEN/ETH', 'NCASH/BTC', 'KAVA/ETH', 'BAL/BTC', 'UNI/ETH', 'Themis/BTC', 'LTC/HT', 'YGG/USDT', 'ELA/ETH', 'NU/USDT', 'SKL/ETH', 'BLZ/BTC', 'CNN/ETH', 'ONT/BTC', 'XRT/ETH', 'PUNDIX/BTC', 'ZRX/ETH', 'ICX/USDT', 'LET/USDT', 'VET/USDT', 'NEXO/ETH', 'LBA/USDT', 'UNI/USDT', 'IRIS/HUSD', 'WPR/ETH', 'BAND/ETH', 'QSP/ETH', 'TRIO/BTC', '1INCH/ETH', 'XRP/HUSD', 'FOR/HT', 'CNNS/USDT', 'DATX/ETH', 'STK/BTC', 'MT/BTC', 'LOL/BTC', 'CHZ/BTC', 'BCH/USDT', 'AE/USDT', 'BCH3S/USDT'

dict_keys(['1INCH-PERP', '1INCH-0924', '1INCH/USD', 'AAPL-0924', 'AAPL/USD', 'AAVE-PERP', 'AAVE-0924', 'AAVE/USD', 'AAVE/USDT', 'ABNB-0924', 'ABNB/USD', 'ACB-0924', 'ACB/USD', 'ADA-PERP', 'ADA-0924', 'AKRO/USD', 'AKRO/USDT', 'ALCX-PERP', 'ALCX/USD', 'ALGO-PERP', 'ALGO-0924', 'ALICE-PERP', 'ALICE/USD', 'ALPHA-PERP', 'ALPHA/USD', 'ALT-PERP', 'ALT-0924', 'AMC-0924', 'AMC/USD', 'AMD-0924', 'AMD/USD', 'AMPL-PERP', 'AMPL/USD', 'AMPL/USDT', 'AMZN-0924', 'AMZN/USD', 'APHA/USD', 'AR-PERP', 'ARKK-0924', 'ARKK/USD', 'ASD-PERP', 'ASD/USD', 'ATLAS-PERP', 'ATLAS/USD', 'ATOM-PERP', 'ATOM-0924', 'AUDIO-PERP', 'AUDIO/USD', 'AUDIO/USDT', 'AVAX-PERP', 'AVAX-0924', 'AXS-PERP', 'AXS/USD', 'BABA-0924', 'BABA/USD', 'BADGER-PERP', 'BADGER/USD', 'BAL-PERP', 'BAL-0924', 'BAL/USD', 'BAL/USDT', 'BAND-PERP', 'BAND/USD', 'BAO-PERP', 'BAO/USD', 'BAT-PERP', 'BAT/USD', 'BB-0924', 'BB/USD', 'BCH-PERP', 'BCH-0924', 'BCH/BTC', 'BCH/USD', 'BCH/USDT', 'BILI-0924', 'BILI/USD', 'BITW-0924', 'BITW/USD', 'BNB-PERP', 'BNB-0924'

dict_keys(['REQ/ETH', 'REQ/BTC', 'NULS/ETH', 'NULS/BTC', 'CVC/BTC', 'NEBL/BTC', 'SNT/ETH', 'SNT/BTC', 'PPT/ETH', 'PPT/BTC', 'BSV/BTC', 'POWR/BTC', 'POWR/ETH', 'BCD/BTC', 'BCD/ETH', 'KNC/ETH', 'KNC/BTC', 'DACC/BTC', 'OMG/ETH', 'OMG/BTC', 'CHSB/BTC', 'CHSB/ETH', 'MANA/BTC', 'BTM/BTC', 'AMB/ETH', 'MANA/ETH', 'AMB/BTC', 'BTM/ETH', 'ETN/BTC', 'DGB/BTC', 'DGB/ETH', 'QTUM/BTC', 'AION/ETH', 'AION/BTC', 'WTC/BTC', 'ENJ/BTC', 'ENJ/ETH', 'SNX/ETH', 'OPEN/ETH', 'TIME/ETH', 'SNX/USDT', 'OPEN/BTC', 'SNX/BTC', 'TIME/BTC', 'XLM/USDT', 'VET/USDT', 'DASH/ETH', 'UTK/BTC', 'UTK/ETH', 'DASH/BTC', 'GO/USDT', 'AXPR/ETH', 'AXPR/BTC', 'ACT/ETH', 'ACT/BTC', 'EOS/KCS', 'ETC/ETH', 'EOS/BTC', 'PLAY/BTC', 'ETC/BTC', 'EOS/USDT', 'EOS/ETH', 'GAS/BTC', 'BCH/BTC', 'PLAY/ETH', 'ETC/USDT', 'DENT/ETH', 'DENT/BTC', 'MWAT/BTC', 'MWAT/ETH', 'DRGN/BTC', 'DRGN/ETH', 'LTC/USDT', 'CV/BTC', 'LTC/ETH', 'CV/ETH', 'LTC/BTC', 'DBC/ETH', 'DBC/BTC', 'KEY/BTC', 'KEY/ETH', 'IOST/ETH', 'ADB/ETH', 'ADB/BTC', 'HPB/BTC', 'HPB/ETH', 'IOST/BTC

dict_keys(['IHT/ETH', 'AME/ETH', 'ALEPH/USDT', 'OGN/USDT', 'HC/USDT', 'SAL/USDT', 'QNT/USDT', 'QTUM/ETH', 'MAHA/ETH', 'POOL/USDT', 'KGC/USDT', 'HARD/USDT', 'FTT/ETH', 'K21/ETH', 'FINE/USDT', 'REP/USDT', 'SKM/ETH', 'QLC/ETH', 'GAS/BTC', 'ALICE3L/USDT', 'BAO/USDT', 'ANT/USDT', 'VIDYX/USDT', 'SMTY/ETH', 'HERO/USDT', 'SHARE/USDT', 'FIN/USDT', 'MTV/USDT', 'JNT/ETH', 'SMTY/USDT', 'ORAO/USDT', 'AE/ETH', 'SUSD/USDT', 'MAN/USDT', 'VET/USDT', 'WAXP/ETH', 'MDA/ETH', 'SPS/USDT', 'STX/ETH', 'WSIENNA/USDT', 'NAOS/BTC', 'NFTX/USDT', 'ICP3L/USDT', 'SFI/ETH', 'BSV3L/USDT', 'DIS/ETH', 'FET/USDT', 'VELO/USDT', 'NSBT/BTC', 'GSE/ETH', 'HNS/BTC', 'BACON/USDT', 'DUSK/USDT', 'EGLD/ETH', 'C983L/USDT', 'FAN/ETH', 'CZZ/USDT', 'SWRV/USDT', 'ONT/ETH', 'KINE/ETH', 'SAKE/ETH', 'SUSD/ETH', 'ONC/USDT', 'VEGA/ETH', 'DAI/USDT', 'PERL/ETH', 'MATIC3S/USDT', 'STMX/USDT', 'SKL/USDT', 'AMPL3S/USDT', 'WEX/USDT', 'ULU/ETH', 'LIKE/ETH', 'INSUR/ETH', 'CAKE/ETH', 'SXP/ETH', 'COTI/USDT', 'AVA/USDT', 'OPA/USDT', 'VEGA/USDT', 'HitCh

### Checking data availability at coinbase

In [130]:
coinbase = log_into_exchange("coinbase")

In [131]:
coinbase.has

{'loadMarkets': True,
 'cancelAllOrders': False,
 'cancelOrder': False,
 'cancelOrders': False,
 'CORS': True,
 'createDepositAddress': True,
 'createLimitOrder': True,
 'createMarketOrder': True,
 'createOrder': False,
 'deposit': False,
 'editOrder': 'emulated',
 'fetchBalance': True,
 'fetchClosedOrders': False,
 'fetchCurrencies': True,
 'fetchDepositAddress': False,
 'fetchDeposits': True,
 'fetchL2OrderBook': False,
 'fetchLedger': True,
 'fetchMarkets': True,
 'fetchMyTrades': False,
 'fetchOHLCV': False,
 'fetchOpenOrders': False,
 'fetchOrder': False,
 'fetchOrderBook': False,
 'fetchOrderBooks': False,
 'fetchOrders': False,
 'fetchOrderTrades': False,
 'fetchStatus': 'emulated',
 'fetchTicker': True,
 'fetchTickers': False,
 'fetchTime': True,
 'fetchTrades': False,
 'fetchTradingFee': False,
 'fetchTradingFees': False,
 'fetchFundingFee': False,
 'fetchFundingFees': False,
 'fetchTradingLimits': False,
 'fetchTransactions': False,
 'fetchWithdrawals': True,
 'privateAPI': T

`coinbase` exchange does not provide any kind of historical data (neither on OHLCV nor on trading orders), and it seems that its API allows only for trading.

## Loading OHLCV data

### Binance

Binance data is being loaded correctly with specified functions.

In [9]:
binance_data = download_ohlcv_data(
    "binance", "2018-01-01T00:00:00Z", "2018-02-01T00:00:00Z", "BTC/USDT"
)

Fetched 500 candles
From 2018-01-01T00:00:00.000Z to 2018-01-01T08:19:00.000Z
Fetched 500 candles in total
Fetched 500 candles
From 2018-01-01T08:20:00.000Z to 2018-01-01T16:39:00.000Z
Fetched 1000 candles in total
Fetched 500 candles
From 2018-01-01T16:40:00.000Z to 2018-01-02T00:59:00.000Z
Fetched 1500 candles in total
Fetched 500 candles
From 2018-01-02T01:00:00.000Z to 2018-01-02T09:19:00.000Z
Fetched 2000 candles in total
Fetched 500 candles
From 2018-01-02T09:20:00.000Z to 2018-01-02T17:39:00.000Z
Fetched 2500 candles in total
Fetched 500 candles
From 2018-01-02T17:40:00.000Z to 2018-01-03T01:59:00.000Z
Fetched 3000 candles in total
Fetched 500 candles
From 2018-01-03T02:00:00.000Z to 2018-01-03T10:19:00.000Z
Fetched 3500 candles in total
Fetched 500 candles
From 2018-01-03T10:20:00.000Z to 2018-01-03T18:39:00.000Z
Fetched 4000 candles in total
Fetched 500 candles
From 2018-01-03T18:40:00.000Z to 2018-01-04T02:59:00.000Z
Fetched 4500 candles in total
Fetched 375 candles
From 2018

Fetched 500 candles
From 2018-01-27T09:20:00.000Z to 2018-01-27T17:39:00.000Z
Fetched 38375 candles in total
Fetched 500 candles
From 2018-01-27T17:40:00.000Z to 2018-01-28T01:59:00.000Z
Fetched 38875 candles in total
Fetched 500 candles
From 2018-01-28T02:00:00.000Z to 2018-01-28T10:19:00.000Z
Fetched 39375 candles in total
Fetched 500 candles
From 2018-01-28T10:20:00.000Z to 2018-01-28T18:39:00.000Z
Fetched 39875 candles in total
Fetched 500 candles
From 2018-01-28T18:40:00.000Z to 2018-01-29T02:59:00.000Z
Fetched 40375 candles in total
Fetched 500 candles
From 2018-01-29T03:00:00.000Z to 2018-01-29T11:19:00.000Z
Fetched 40875 candles in total
Fetched 500 candles
From 2018-01-29T11:20:00.000Z to 2018-01-29T19:39:00.000Z
Fetched 41375 candles in total
Fetched 500 candles
From 2018-01-29T19:40:00.000Z to 2018-01-30T03:59:00.000Z
Fetched 41875 candles in total
Fetched 500 candles
From 2018-01-30T04:00:00.000Z to 2018-01-30T12:19:00.000Z
Fetched 42375 candles in total
Fetched 500 candles

### Huobi

For Huobi we see that the data is starting to be loaded from incorrect and a very recent time period that does not belong to the specified time range.

In [133]:
huobi_data = download_ohlcv_data(
    "huobi", "2021-01-01T00:00:00Z", "2021-02-01T00:00:00Z", "BTC/USDT"
)

Fetched 500 candles
From 2021-09-03T05:31:00.000Z to 2021-09-03T13:50:00.000Z
Fetched 500 candles in total
Fetched 500 candles
From 2021-09-03T05:31:00.000Z to 2021-09-03T13:50:00.000Z
Fetched 1000 candles in total
Fetched 500 candles
From 2021-09-03T05:32:00.000Z to 2021-09-03T13:51:00.000Z
Fetched 1500 candles in total


KeyboardInterrupt: 

The reason behind it is that Huobi outputs candles in reverced order.<br>
To demonstrate it let's run `fetch_ohlcv()` for Huobi with differnt limits.

In [48]:
huobi_exchange = log_into_exchange("huobi")
start_date = huobi_exchange.parse8601("2021-01-01T00:00:00Z")
limit1 = 500
limit2 = 1000
limit3 = 2000

In [49]:
huobi_all_data1 = huobi_exchange.fetch_ohlcv("BTC/USDT", "1m", start_date, limit1)
huobi_all_data2 = huobi_exchange.fetch_ohlcv("BTC/USDT", "1m", start_date, limit2)
huobi_all_data3 = huobi_exchange.fetch_ohlcv("BTC/USDT", "1m", start_date, limit3)

The amount of loaded candles is equal to the specified limit, so the data is available.

In [50]:
print(len(huobi_all_data1))
print(len(huobi_all_data2))
print(len(huobi_all_data3))

500
1000
2000


However, if we take a look at the timestamps of the first and the last candles, we see that the last candle is always the most recent one while the first one is equal to the most recent candle's timestamp munis the rime range specified in the request.

In [41]:
huobi_first_candle_date1 = huobi_exchange.iso8601(huobi_all_data1[0][0])
print(huobi_first_candle_date1)
#
huobi_first_candle_date2 = huobi_exchange.iso8601(huobi_all_data2[0][0])
print(huobi_first_candle_date2)
#
huobi_first_candle_date3 = huobi_exchange.iso8601(huobi_all_data3[0][0])
print(huobi_first_candle_date3)

2021-09-03T04:11:00.000Z
2021-09-02T19:51:00.000Z
2021-09-02T03:11:00.000Z


In [42]:
huobi_last_candle_date1 = huobi_exchange.iso8601(huobi_all_data1[-1][0])
print(huobi_last_candle_date1)
#
huobi_last_candle_date2 = huobi_exchange.iso8601(huobi_all_data2[-1][0])
print(huobi_last_candle_date2)
#
huobi_last_candle_date3 = huobi_exchange.iso8601(huobi_all_data3[-1][0])
print(huobi_last_candle_date3)

2021-09-03T12:30:00.000Z
2021-09-03T12:30:00.000Z
2021-09-03T12:30:00.000Z


This is confirmed by calculation of steps between the last and first candles timestamps

In [60]:
print((huobi_all_data1[-1][0] - huobi_all_data1[0][0]) / 60 / 1000 + 1)
print((huobi_all_data2[-1][0] - huobi_all_data2[0][0]) / 60 / 1000 + 1)
print((huobi_all_data3[-1][0] - huobi_all_data3[0][0]) / 60 / 1000 + 1)

500.0
1000.0
2000.0


Therefore, it seems that Huobi data can be loaded, but additional research is needed to understand, how to do it correctly and it definitely needs a 'personal' approach.

### FTX

FTX data is being loaded correctly, we have it starting from 2020-03-28 14:40:00

In [76]:
ftx_data = download_ohlcv_data(
    "ftx", "2020-03-28T00:00:00Z", "2020-04-01T00:00:00Z", "BTC/USDT"
)

Fetched 0 candles
Fetched 0 candles in total
Fetched 121 candles
From 2020-03-28T14:40:00.000Z to 2020-03-28T16:40:00.000Z
Fetched 121 candles in total
Fetched 500 candles
From 2020-03-28T16:40:00.000Z to 2020-03-29T00:59:00.000Z
Fetched 621 candles in total
Fetched 500 candles
From 2020-03-29T01:00:00.000Z to 2020-03-29T09:19:00.000Z
Fetched 1121 candles in total
Fetched 500 candles
From 2020-03-29T09:20:00.000Z to 2020-03-29T17:39:00.000Z
Fetched 1621 candles in total
Fetched 500 candles
From 2020-03-29T17:40:00.000Z to 2020-03-30T01:59:00.000Z
Fetched 2121 candles in total
Fetched 500 candles
From 2020-03-30T02:00:00.000Z to 2020-03-30T10:19:00.000Z
Fetched 2621 candles in total
Fetched 500 candles
From 2020-03-30T10:20:00.000Z to 2020-03-30T18:39:00.000Z
Fetched 3121 candles in total
Fetched 500 candles
From 2020-03-30T18:40:00.000Z to 2020-03-31T02:59:00.000Z
Fetched 3621 candles in total
Fetched 500 candles
From 2020-03-31T03:00:00.000Z to 2020-03-31T11:19:00.000Z
Fetched 4121 ca

### Kucoin

Kucoin data is being loaded correctly as well.

In [134]:
kucoin_data2021 = download_ohlcv_data(
    "kucoin", "2021-01-01T00:00:00Z", "2021-01-04T00:00:00Z", "BTC/USDT"
)

Fetched 500 candles
From 2021-01-01T00:00:00.000Z to 2021-01-01T08:19:00.000Z
Fetched 500 candles in total
Fetched 500 candles
From 2021-01-01T08:20:00.000Z to 2021-01-01T16:39:00.000Z
Fetched 1000 candles in total
Fetched 500 candles
From 2021-01-01T16:40:00.000Z to 2021-01-02T00:59:00.000Z
Fetched 1500 candles in total
Fetched 500 candles
From 2021-01-02T01:00:00.000Z to 2021-01-02T09:19:00.000Z
Fetched 2000 candles in total
Fetched 500 candles
From 2021-01-02T09:20:00.000Z to 2021-01-02T17:39:00.000Z
Fetched 2500 candles in total
Fetched 500 candles
From 2021-01-02T17:40:00.000Z to 2021-01-03T01:59:00.000Z
Fetched 3000 candles in total
Fetched 500 candles
From 2021-01-03T02:00:00.000Z to 2021-01-03T10:19:00.000Z
Fetched 3500 candles in total
Fetched 500 candles
From 2021-01-03T10:20:00.000Z to 2021-01-03T18:39:00.000Z
Fetched 4000 candles in total
Fetched 500 candles
From 2021-01-03T18:40:00.000Z to 2021-01-04T02:59:00.000Z
Fetched 4500 candles in total


However, the amount of candles for earlier periods is unstable and timestamps are slipping. Additional research is required to understand if the data is really missing or its some sort of a bug.

In [135]:
kucoin_data2018 = download_ohlcv_data(
    "kucoin", "2018-01-01T00:00:00Z", "2018-01-04T00:00:00Z", "BTC/USDT"
)

Fetched 95 candles
From 2018-01-01T00:03:00.000Z to 2018-01-01T08:18:00.000Z
Fetched 95 candles in total
Fetched 98 candles
From 2018-01-01T08:27:00.000Z to 2018-01-01T16:34:00.000Z
Fetched 193 candles in total
Fetched 123 candles
From 2018-01-01T16:41:00.000Z to 2018-01-02T00:59:00.000Z
Fetched 316 candles in total
Fetched 184 candles
From 2018-01-02T01:00:00.000Z to 2018-01-02T09:18:00.000Z
Fetched 500 candles in total
Fetched 157 candles
From 2018-01-02T09:21:00.000Z to 2018-01-02T17:39:00.000Z
Fetched 657 candles in total
Fetched 252 candles
From 2018-01-02T17:40:00.000Z to 2018-01-03T01:56:00.000Z
Fetched 909 candles in total
Fetched 168 candles
From 2018-01-03T02:07:00.000Z to 2018-01-03T10:16:00.000Z
Fetched 1077 candles in total
Fetched 139 candles
From 2018-01-03T10:21:00.000Z to 2018-01-03T18:38:00.000Z
Fetched 1216 candles in total
Fetched 241 candles
From 2018-01-03T18:40:00.000Z to 2018-01-04T02:57:00.000Z
Fetched 1457 candles in total


### Bitfinex

The Bitfinex candles are being loaded with overlapping time periods.

In [94]:
bitfinex_data = download_ohlcv_data(
    "bitfinex", "2020-01-01T00:00:00Z", "2020-02-01T00:00:00Z", "BTC/USDT"
)

Fetched 500 candles
From 2020-01-01T00:00:00.000Z to 2020-01-02T17:14:00.000Z
Fetched 500 candles in total
Fetched 500 candles
From 2020-01-01T08:21:00.000Z to 2020-01-02T20:22:00.000Z
Fetched 1000 candles in total
Fetched 500 candles
From 2020-01-01T16:55:00.000Z to 2020-01-03T01:13:00.000Z
Fetched 1500 candles in total
Fetched 500 candles
From 2020-01-02T01:07:00.000Z to 2020-01-03T04:40:00.000Z
Fetched 2000 candles in total
Fetched 500 candles
From 2020-01-02T09:25:00.000Z to 2020-01-03T08:44:00.000Z
Fetched 2500 candles in total


KeyboardInterrupt: 

To research this let's check the amount of minute steps between the last candle timestamp and the first one.

In [88]:
bitfinex_exchange = log_into_exchange("bitfinex")
start_date = bitfinex_exchange.parse8601("2021-01-01T00:00:00Z")
limit1 = 500
limit2 = 1000
limit3 = 2000

In [89]:
bitfinex_all_data1 = bitfinex_exchange.fetch_ohlcv(
    "BTC/USDT", "1m", start_date, limit1
)
bitfinex_all_data2 = bitfinex_exchange.fetch_ohlcv(
    "BTC/USDT", "1m", start_date, limit2
)
bitfinex_all_data3 = bitfinex_exchange.fetch_ohlcv(
    "BTC/USDT", "1m", start_date, limit3
)

In [90]:
print(len(bitfinex_all_data1))
print(len(bitfinex_all_data2))
print(len(bitfinex_all_data3))

500
1000
2000


In [91]:
bitfinex_first_candle_date1 = bitfinex_exchange.iso8601(bitfinex_all_data1[0][0])
print(bitfinex_first_candle_date1)
#
bitfinex_first_candle_date2 = bitfinex_exchange.iso8601(bitfinex_all_data2[0][0])
print(bitfinex_first_candle_date2)
#
bitfinex_first_candle_date3 = bitfinex_exchange.iso8601(bitfinex_all_data3[0][0])
print(bitfinex_first_candle_date3)

2021-01-01T00:00:00.000Z
2021-01-01T00:00:00.000Z
2021-01-01T00:00:00.000Z


In [92]:
bitfinex_last_candle_date1 = bitfinex_exchange.iso8601(bitfinex_all_data1[-1][0])
print(bitfinex_last_candle_date1)
#
bitfinex_last_candle_date2 = bitfinex_exchange.iso8601(bitfinex_all_data2[-1][0])
print(bitfinex_last_candle_date2)
#
bitfinex_last_candle_date3 = bitfinex_exchange.iso8601(bitfinex_all_data3[-1][0])
print(bitfinex_last_candle_date3)

2021-01-01T11:07:00.000Z
2021-01-01T22:29:00.000Z
2021-01-02T20:00:00.000Z


Turns out that the amount of minute steps betweent the last and the firt candles is higher than the specified limits. One possible reason is that Bitfinex has gaps in its data. However, if this is true, why the behaviour for Kucoin was correct since there is a clear gap in data for it in 2018?
Thus, this should be researched.

In [93]:
print((bitfinex_all_data1[-1][0] - bitfinex_all_data1[0][0]) / 60 / 1000 + 1)
print((bitfinex_all_data2[-1][0] - bitfinex_all_data2[0][0]) / 60 / 1000 + 1)
print((bitfinex_all_data3[-1][0] - bitfinex_all_data3[0][0]) / 60 / 1000 + 1)

668.0
1350.0
2641.0


Still, we see that Bitfinex has data and it is more than possible to extract it. Here we wanted to develop a unified approach for loading data from different exchanges. If this doesn't work for Bitfinex, there is and example guide in CCXT library https://github.com/ccxt/ccxt/blob/master/examples/py/fetch-bitfinex-ohlcv-history.py that we can use to develop a 'personal' approach to this exchange. So we're good here.

### Gateio

Gateio data is being loaded correctly, we have it starting from 2021-06-05 13:33:00

In [136]:
gateio_data = download_ohlcv_data(
    "gateio", "2021-06-05T00:00:00Z", "2021-06-10T00:00:00Z", "BTC/USDT"
)

Fetched 0 candles
Fetched 0 candles in total
Fetched 168 candles
From 2021-06-05T13:52:00.000Z to 2021-06-05T16:39:00.000Z
Fetched 168 candles in total
Fetched 500 candles
From 2021-06-05T16:40:00.000Z to 2021-06-06T00:59:00.000Z
Fetched 668 candles in total
Fetched 500 candles
From 2021-06-06T01:00:00.000Z to 2021-06-06T09:19:00.000Z
Fetched 1168 candles in total
Fetched 500 candles
From 2021-06-06T09:20:00.000Z to 2021-06-06T17:39:00.000Z
Fetched 1668 candles in total
Fetched 500 candles
From 2021-06-06T17:40:00.000Z to 2021-06-07T01:59:00.000Z
Fetched 2168 candles in total
Fetched 500 candles
From 2021-06-07T02:00:00.000Z to 2021-06-07T10:19:00.000Z
Fetched 2668 candles in total
Fetched 500 candles
From 2021-06-07T10:20:00.000Z to 2021-06-07T18:39:00.000Z
Fetched 3168 candles in total
Fetched 500 candles
From 2021-06-07T18:40:00.000Z to 2021-06-08T02:59:00.000Z
Fetched 3668 candles in total
Fetched 500 candles
From 2021-06-08T03:00:00.000Z to 2021-06-08T11:19:00.000Z
Fetched 4168 ca

### Kraken

Kraken provides only 720 of candles into the past for any selected time step. (see https://github.com/ccxt/ccxt/issues/8091#issuecomment-739600165)<br>
We can track its data further on but there is no way to get its historical data further than that.

In [123]:
kraken_exchange = log_into_exchange("kraken")
start_date = kraken_exchange.parse8601("2020-01-01T00:00:00Z")
kraken_all_data = kraken_exchange.fetch_ohlcv("BTC/USDT", "1m", start_date, 1000)

In [124]:
len(kraken_all_data)

720

The first and the last candles timstamps are being updated all the time, no matter which start date you specify in the query.

In [125]:
kraken_first_candle_date = kraken_exchange.iso8601(kraken_all_data[0][0])
kraken_first_candle_date

'2021-09-03T01:39:00.000Z'

In [126]:
kraken_last_candle_date = kraken_exchange.iso8601(kraken_all_data[-1][0])
kraken_last_candle_date

'2021-09-03T13:38:00.000Z'

## Summary

- We can apply sort of a unified approach to load all the historical data for Binance, FTX, Kucoin, and Gateio. Although, Kucoin case should be investigated a bit more in order to confirm that there is no bug.
- Huobi and Bitfinex seem to have the data to provide but they require a 'personal' approaches that need to be developed.
- Kraken provides only 720 last candles for a specified time step. We may decide to track it from acertain moment or just forget about it in terms of historical data.
- Coinbase exchange does not provide any kind of historical data.